# US Air Freight and Waterway Volumes

---

A look at air and domestic waterway freight volumes, according to the US DOT Bureau of Transportation Statistics.

In [1]:
import pandas as pd
import altair as alt
from os import environ
import re

#if environ.get("GITHUB_WORKFLOW"):
#    raise RuntimeError("Requires manual updates")

In [2]:
def doComboChart(df, metric='AIR_RTMFM', title='US Air Freight Volume', label='Volume [ton-miles]', width=2):

    brush = alt.selection(type='interval', encodings=['x'])
    
    base = alt.Chart(df.reset_index()[['dt', metric]].dropna()).mark_line(color='purple').encode(
        alt.X('dt:T', axis=alt.Axis(title='')),
        alt.Y(f'{metric}:Q', axis=alt.Axis(title=label)),
        tooltip=[alt.Tooltip("dt:T", format="%b %Y"), alt.Tooltip(f"{metric}:Q", format=",.02f")]
    ).properties(
        width=750,
        height=400
    )

    upper = base.mark_bar(width=width, color='blue').transform_window(
        window=[alt.WindowFieldDef("lag", metric, param=12, **{ 'as': 'previous' })],
        sort=[{"field": "dt"}],
    ).transform_calculate(
        yoy=f"((datum['{metric}'] - datum.previous) / datum.previous) * 100"
    ).encode(
        alt.X('dt:T', axis=alt.Axis(title=''), scale=alt.Scale(domain=brush), impute=alt.ImputeParams(value=None)),
        alt.Y(f'yoy:Q', axis=alt.Axis(title='Year-over-Year Growth [%]'), impute=alt.ImputeParams(method='value', keyvals=[400, 1000])),
        color=alt.condition(f"datum['yoy'] < 0",
            alt.value('lightsalmon'),
            alt.value('royalblue')
        )
    ).properties(
        title=title,
    )
    
    lower = base.properties(
        height=100
    ).add_selection(brush)

    return (upper & lower).properties(background='white')

In [3]:
# https://data.bts.gov/Research-and-Statistics/Transportation-Services-Index-and-Seasonally-Adjus/bw6n-ddqk

df_af = pd.read_csv('https://data.bts.gov/api/views/bw6n-ddqk/rows.csv?accessType=DOWNLOAD')
df_af = df_af.iloc[:-1, 1:]
df_af.columns = ['dt'] + df_af.columns[1:].tolist()
#df_af.head()

## Airline Freight

In [4]:
c = doComboChart(df_af)
c.save('transportation-air-waterway.png')
c.display()

alt.VConcatChart(...)

## Internal Waterways Shipping

In [5]:
doComboChart(df_af, metric='WATERBORNE', title='US Internal Waterways Shipping Volume', label='Volume [Million short tons]')

alt.VConcatChart(...)

## Rail Freight Carloads

In [6]:
doComboChart(df_af, metric='RAIL_FRT_CARLOADS', title='US Rail Freight Carloads', label='Carloads')

alt.VConcatChart(...)

## Rail Freight Intermodal

In [7]:
doComboChart(df_af, metric='RAIL_FRT_INTERMODAL', title='US Rail Freight Intermodal', label='Carloads')

alt.VConcatChart(...)

## Vehicle Miles Traveled

In [8]:
doComboChart(df_af, metric='VMT', title='US Vehicle Miles Travels', label='Miles')

alt.VConcatChart(...)